## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
from matplotlib import pyplot as plt
import os, shutil
import random
import cv2
from __future__ import print_function
import pickle
import time
import glob
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.layers import Dropout
from sklearn.externals import joblib
import argparse
from keras.callbacks import *
import sys
import theano
import theano.tensor as T
from keras import initializers
from keras.layers import BatchNormalization
import copy
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from tensorflow.keras.models import load_model as load_initial_model
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import binary_crossentropy
import gc
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout
from keras.layers import concatenate, Conv2DTranspose, BatchNormalization
from keras import backend as K
from tensorflow.keras.metrics import AUC


## Merge Augmentations

In [ ]:
def merge_augmentations(augment_dir, output_dir, list_of_aug_files):
  '''
    - augment_dir         (string)   : The path which has subfolders that are augmentation folders.
    - output_dir          (string)   : The path you want to put all augmentations.
    - list_of_aug_files   (list)     : List of names which contains augmentations.
  '''
  os.mkdir(output_dir + '/images')
  os.mkdir(output_dir + '/labels')

  for folder in list_of_aug_files:
    if folder != 'train':
      for file in sorted(os.listdir(augment_dir + '/' + folder + '/images')):
        shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + folder + '_' + file.split("_")[-1].split('.')[0] + '.png')
        shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + folder + '_' + file.split("_")[-1].split('.')[0] + '.png')
    else:
      for file in sorted(os.listdir(augment_dir + '/' + folder + '/images')):
        shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file.split("_")[-1].split('.')[0] + '.png')
        shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file.split("_")[-1].split('.')[0] + '.png')
    

    print(folder + ' folder has been merged...')
    print('Number of images in output: ' + str(len(os.listdir(output_dir + '/images'))))
 

  print('Merging is done successfully!')

## Handcrafted Metrics

In [ ]:
def dice_coef(y_true, y_pred):
  smooth = 0.0
  y_true_f = keras.flatten(y_true)
  y_pred_f = keras.flatten(y_pred)
  intersection = keras.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + smooth)

def jacard(y_true, y_pred):

  y_true_f = keras.flatten(y_true)
  y_pred_f = keras.flatten(y_pred)
  intersection = keras.sum ( y_true_f * y_pred_f)
  union = keras.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

  return intersection/union

## Generators

In [ ]:
def adjustData(img,mask,flag_multi_class,num_class):
  if(flag_multi_class):
    img = img / 255
    mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
    new_mask = np.zeros(mask.shape + (num_class,))
    for i in range(num_class):
        #for one pixel in the image, find the class in mask and convert it into one-hot vector
        #index = np.where(mask == i)
        #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
        #new_mask[index_mask] = 1
        new_mask[mask == i,i] = 1
    new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
    mask = new_mask
  elif (np.max(img) > 1):
    img = img / 255
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
  return (img,mask)

In [ ]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (608,704),seed = 1):
  image_datagen = ImageDataGenerator(**aug_dict)
  mask_datagen = ImageDataGenerator(**aug_dict)
  image_generator = image_datagen.flow_from_directory(
      train_path,
      classes = [image_folder],
      class_mode = None,
      color_mode = image_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = image_save_prefix,
      seed = seed)
  mask_generator = mask_datagen.flow_from_directory(
      train_path,
      classes = [mask_folder],
      class_mode = None,
      color_mode = mask_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = mask_save_prefix,
      seed = seed)
  train_generator = zip(image_generator, mask_generator)
  for (img,mask) in train_generator:
    img,mask = adjustData(img,mask,flag_multi_class,num_class)
    yield (img,mask)

In [ ]:
def testGenerator2(batch_size,test_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (608,704),seed = 1):
  image_datagen = ImageDataGenerator(**aug_dict)
  mask_datagen = ImageDataGenerator(**aug_dict)
  image_generator = image_datagen.flow_from_directory(
      test_path,
      classes = [image_folder],
      class_mode = None,
      color_mode = image_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = image_save_prefix,
      seed = seed)
  mask_generator = mask_datagen.flow_from_directory(
      test_path,
      classes = [mask_folder],
      class_mode = None,
      color_mode = mask_color_mode,
      target_size = target_size,
      batch_size = batch_size,
      save_to_dir = save_to_dir,
      save_prefix  = mask_save_prefix,
      seed = seed)
  test_generator = zip(image_generator, mask_generator)
  for (img,mask) in test_generator:
    img,mask = adjustData(img,mask,flag_multi_class,num_class)
    yield (img,mask)

## Model

In [ ]:
def fire_module(x, fire_id, squeeze=16, expand=64):
    f_name = "fire{0}/{1}"
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(squeeze, (1, 1), activation='relu', padding='same', name=f_name.format(fire_id, "squeeze1x1"))(x)
    x = BatchNormalization(axis=channel_axis)(x)

    left = Conv2D(expand, (1, 1), activation='relu', padding='same', name=f_name.format(fire_id, "expand1x1"))(x)
    right = Conv2D(expand, (3, 3), activation='relu', padding='same', name=f_name.format(fire_id, "expand3x3"))(x)
    x = concatenate([left, right], axis=channel_axis, name=f_name.format(fire_id, "concat"))
    return x

def SqueezeUNet(inputs, num_classes=None, deconv_ksize=3, dropout=0.5, activation='sigmoid'):
    """SqueezeUNet is a implementation based in SqueezeNetv1.1 and unet for semantic segmentation
    :param inputs: input layer.
    :param num_classes: number of classes.
    :param deconv_ksize: (width and height) or integer of the 2D deconvolution window.
    :param dropout: dropout rate
    :param activation: type of activation at the top layer.
    :returns: SqueezeUNet model
    """
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    if num_classes is None:
        num_classes = K.int_shape(inputs)[channel_axis]

    x01 = Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu', name='conv1')(inputs)
    x02 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1', padding='same')(x01)

    x03 = fire_module(x02, fire_id=2, squeeze=16, expand=64)
    x04 = fire_module(x03, fire_id=3, squeeze=16, expand=64)
    x05 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3', padding="same")(x04)

    x06 = fire_module(x05, fire_id=4, squeeze=32, expand=128)
    x07 = fire_module(x06, fire_id=5, squeeze=32, expand=128)
    x08 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5', padding="same")(x07)

    x09 = fire_module(x08, fire_id=6, squeeze=48, expand=192)
    x10 = fire_module(x09, fire_id=7, squeeze=48, expand=192)
    x11 = fire_module(x10, fire_id=8, squeeze=64, expand=256)
    x12 = fire_module(x11, fire_id=9, squeeze=64, expand=256)

    if dropout != 0.0:
        x12 = Dropout(dropout)(x12)

    up1 = concatenate([
        Conv2DTranspose(192, deconv_ksize, strides=(1, 1), padding='same')(x12),
        x10,
    ], axis=channel_axis)
    up1 = fire_module(up1, fire_id=10, squeeze=48, expand=192)

    up2 = concatenate([
        Conv2DTranspose(128, deconv_ksize, strides=(1, 1), padding='same')(up1),
        x08,
    ], axis=channel_axis)
    up2 = fire_module(up2, fire_id=11, squeeze=32, expand=128)

    up3 = concatenate([
        Conv2DTranspose(64, deconv_ksize, strides=(2, 2), padding='same')(up2),
        x05,
    ], axis=channel_axis)
    up3 = fire_module(up3, fire_id=12, squeeze=16, expand=64)

    up4 = concatenate([
        Conv2DTranspose(32, deconv_ksize, strides=(2, 2), padding='same')(up3),
        x02,
    ], axis=channel_axis)
    up4 = fire_module(up4, fire_id=13, squeeze=16, expand=32)
    up4 = UpSampling2D(size=(2, 2))(up4)

    x = concatenate([up4, x01], axis=channel_axis)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(num_classes, (1, 1), activation=activation)(x)
    model = Model(inputs, x)
    #model.summary()
    model.compile(optimizer=Adam(lr = 1e-4), loss=binary_crossentropy, metrics = ['accuracy',dice_coef,jacard,tf.keras.metrics.MeanIoU(num_classes=2),AUC(name='auc'),tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])                                                                                
    return model

## Random Search

In [ ]:
input_path        = "/content/drive/MyDrive/png_data"
output_path       = "/content/drive/MyDrive/data/random_search"
test_path         = "/content/drive/MyDrive/png_data/train" # you have to split your data train and test  

n_iteration       = 10
n_augmentation    = 85
epoch             = 6

input_height      = 608
input_width       = 704
base_sample_count = 20
exclusion_list = ['.ipynb_checkpoints','combinations','test']

In [ ]:
def random_search(input_path, output_path, test_path, n_iteration, n_augmentation, base_sample_count, epoch, input_height, input_width, exclusion_list):
  
  max_index = 0
  list_of_picked_augs = []
  dice_coefs = []
  aucs = []
  accuracies = []
  aug_list = os.listdir(input_path)
  
  print("Merge and Train :\n\n")
  for i in range(n_iteration):
    
    augmentations=[]
    exclude = exclusion_list
    
    os.mkdir(output_path + "/{}".format(i))
    new_output_path = output_path + "/{}".format(i)
    
    for m,n in enumerate(range(n_augmentation)):
      aug2=[i for i in aug_list if i not in augmentations and i not in exclude]
      random_aug=random.choice(aug2)
      augmentations.append(random_aug)
    
    print(f"Picked augmentations for {i}th iteration: ")
    print(augmentations)
    print('\n')
    list_of_picked_augs.append(augmentations)
    merge_augmentations(input_path, new_output_path, augmentations)
    print('------------------------------------------------------------------------------')

    print(f'Training for {i}th choices:\n')
    os.mkdir(output_path+'/'+str(i)+'/logs')
    os.mkdir(output_path+'/'+str(i)+'/checkpoints')
    
    LOG_PATH        = output_path+'/'+str(i)+'/logs'
    CKPTS_PATH      = output_path+'/'+str(i)+'/checkpoints'

    data_gen_args = dict()
    train_generator = trainGenerator(2, output_path + '/' + str(i), 'images','labels', data_gen_args, save_to_dir = None, target_size=(input_height,input_width))
    test_generator = testGenerator2(2, test_path, 'images','labels', data_gen_args, save_to_dir = None, target_size=(input_height,input_width))

    model = SqueezeUNet(inputs=Input((input_height, input_width, 1)))
    model_checkpoint = ModelCheckpoint(CKPTS_PATH + "/squeezeunet.hdf5", monitor='loss',verbose=1, save_best_only=True)
    model_history = model.fit_generator(train_generator,steps_per_epoch=n_augmentation*base_sample_count//2,epochs=epoch,callbacks=[model_checkpoint],
                                        validation_data=test_generator, validation_steps=len(os.listdir(test_path + '/images'))//2)

    log_file = open(LOG_PATH + "/log.pkl", "wb")#history file
    pickle.dump(model_history.history, log_file)
    log_file.close()

    dice_coefs.append(model_history.history['val_dice_coef'][epoch-1])
    aucs.append(model_history.history['val_auc'][epoch-1])
    accuracies.append(model_history.history['val_accuracy'][epoch-1])



    del model
    gc.collect()
    print('------------------------------------------------------------------------------')

    for i,item in enumerate(aucs):
      print(f'For {i}th choices :\t{item}')
      if item > aucs[max_index]:
        max_index = i
    
    print('------------------------------------------------------------------------------')
    print(f'Best Result for now occured {max_index}th choices and it is :\t{aucs[max_index]}, \nwith accuracy:\t{accuracies[max_index]}, \nwith dice coef:\t{dice_coefs[max_index]} \nPicked Augmentations :\n{list_of_picked_augs[max_index]}')
    print('--------------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
random_search(input_path, output_path, test_path, n_iteration, n_augmentation, base_sample_count, epoch, input_height, input_width, exclusion_list)

Merge and Train :


Picked augmentations for 0th iteration: 
['shifty_100_180', 'rotation300', 'eq_hist', 'shearY', 'shearX_90', 'gamma_corr05_rotation300', 'dropout010', 'album_heavy180', 'album_medium', 'white_noise10_90', 'style_transfer', 'flipping1_rotat60', 'zoom_out08_rotat330', 'album_optical90', 'rotation90', 'flipping0_rotat180', 'blurring5_270', 'shifty_-100', 'flipping0_rotat150', 'blurring3', 'zoom_out08_rotat150', 'album_random_crop3', 'album_optical270', 'zoom_out07', 'gamma_corr05_rotation120', 'shiftx_-100_180', 'shearing03', 'zoom_out08_rotat30', 'flipping1_rotat270', 'flipping1_rotat180', 'dropout010_180', 'album_optical180', 'dropout010_270', 'shifty_-100_90', 'shifty_100', 'dropout010_90', 'rotation150', 'flipping0_rotat210', 'rotation60', 'album_elastic', 'sharpen_270', 'zoom_out08_rotat300', 'gamma_corr05_rotation240', 'shiftx_-100_270', 'sharpen_90', 'shifty_100_90', 'rotation30', 'flipping0_rotat330', 'album_heavy', 'flipping1', 'album_random_crop4', 'flipping0

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 1700 images belonging to 1 classes.
Found 1700 images belonging to 1 classes.
Epoch 1/6
850/850 [==============================] - ETA: 0s - loss: 0.2551 - accuracy: 0.9071 - dice_coef: 0.2363 - jacard: 0.1369 - mean_io_u_14: 0.4545 - auc: 0.8033 - precision_14: 0.4304 - recall_14: 0.0974Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
850/850 [==============================] - 150s 172ms/step - loss: 0.2550 - accuracy: 0.9071 - dice_coef: 0.2364 - jacard: 0.1369 - mean_io_u_14: 0.4545 - auc: 0.8034 - precision_14: 0.4307 - recall_14: 0.0976 - val_loss: 0.1628 - val_accuracy: 0.9386 - val_dice_coef: 0.4509 - val_jacard: 0.2917 - val_mean_io_u_14: 0.4569 - val_auc: 0.9281 - val_precision_14: 0.7041 - val_recall_14: 0.4971

Epoch 00001: loss improved from inf to 0.21533, saving model to /content/drive/MyDrive/data/random_search/0/checkpoints/squeezeunet.hdf5
Epoch 2/6
850/850 [==============================] - 148s 174ms/step - loss: 0.1781 - accuracy